# Data Set (Training and Test set)

We used IMDb Dataset which can be downloaded from [here](https://ai.stanford.edu/~amaas/data/sentiment/).
This data set contains 50,000 reviews which is evenly split into two groups: 25,000 reviews for each of training and testing. The reviews for training and testing data sets contain a disjoint set of movies. Therefore, we can assume that the validation result with testing data set can be applicable for other movie reviews.

Each group has the same number of positive and negative reviews: a positive review has a score from 7 to 10 while a negative review has a score from 1 to 4. The reviews having score 5 or 6 are excluded to avoid vagueness.


# Environment

For this project, we used my own Linux machine having AMD Ryzen 7 2700X, 16GB Memory, Geforce RTX 2070.
In addition, Keras with Tensorflow backend is used for making a deep learning model.

In [1]:
from string import punctuation
import os 
#from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
#
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
#
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend.tensorflow_backend as K
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import string
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import glob
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from nltk.stem.porter import PorterStemmer
from collections import Counter
from operator import itemgetter
import numpy as np
from multiprocessing import Pool
import sys

from IPython.display import HTML, display
import tabulate
import pandas as pd
from keras_tqdm import TQDMNotebookCallback


Using TensorFlow backend.


# Tensorflow initial setup

To allow Tensorflow to use enough GPU memory, *allow_growth* option is turned on.

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

# Loading data set files

First of all, all the documents are loaded. The data sets for training and testing are stored in *data/train* and *data/test*, respectively. For each data set, positive and negative reviews are stored in *pos* and *neg* sub-directories.

I have attached the progress bars using the [tqdm](https://github.com/tqdm/tqdm), which is useful in dealting with large data by allowing us to estimate each time of the stages.

Referenced article for tqdm: https://towardsdatascience.com/progress-bars-in-python-4b44e8a4c482

In [3]:
# load all docs in a directory
def load_docs(directory):
    documents = list()
    # walk through all files in the folder
    for filename in tqdm(os.listdir(directory)):
        # create the full path of the file to open
        path = directory + '/' + filename
        with open(path, 'r') as f:
            # load the doc
            doc = f.read()
            # add to list
            documents.append(doc)
    return documents

# load all training reviews
print("Loading training-positive-docs")
global_train_positive_docs = load_docs('data/train/pos')
print("Loading training-negative-docs")
global_train_negative_docs = load_docs('data/train/neg')
# load all test reviews
print("Loading test-positive-docs")
global_test_positive_docs = load_docs('data/test/pos')
print("Loading test-negative-docs")
global_test_negative_docs = load_docs('data/test/neg')

Loading training-positive-docs



Loading training-negative-docs



Loading test-positive-docs



Loading test-negative-docs


# Cleaning documents

### Pre-processing techniques

In most of NLP releated works, documents are normally pre-processed to get better performance.
We tried to apply several techniques which are well-known as follows:

**1. Removing punctuations**  
Normally punctuations do not have any meaning, but they exist for understandability. Therefore, such punctuations should be removed. But, we did not remove the apostrophe mark (') since such removing caused the incorrect stemming.

**2. Removing stopwords**  
We filtered out the stopwords.
The stop words are those words that do not contribute to the deeper meaning of the phrase.
They are the most common words such as: “the“, “a“, and “is“.
NLTK provides a list of commonly agreed upon stop words for a variety of languages.

**3. Stemming**    
The *PorterStemmer* is provided in *NLTK python package*.
We made the words into lowercases and used the stemming method in order to both reduce the vocabulary and to focus on the sense or sentiment of a document rather than deeper meaning.

**4. Removing non-frequent words**   
It is important to define a vocabulary of known words when using a bag-of-words or embedding model.
The more words, the larger the representation of documents, therefore it is important to constrain the words to only those believed to be predictive. 

In this project, **we set up the vocabulary dictionary by removing the non-frequent words to prevent a model from overfitting.** 
This is implemeted in [*vocab.ipynb*](https://github.com/ahrimhan/data-science-project/tree/master/project2/src/vocab.ipynb).

* After removing all words that have a length <= 1 character, we first construct the vocabulary dictionary based on only reviews in the training dataset (Number of vocabularies: 52,826).
* Then, we iterate the vocabulary dictionary again for counting the word occurrences and removing the non-frequent words that have a low occurrence, such as only being used once or none. Thus, remaining vocabularies have the two or more occurrences (Number of filtered vocabularies: 30,819). These filtered vocabularies are saved in [*vocab.txt*](https://github.com/ahrimhan/data-science-project/tree/master/project2/src/vocab.txt).

In [4]:
remove_punctuation_table = str.maketrans('', '', '\'"!.,?:;')
stop_words = set(stopwords.words('english'))
# turn a doc into clean tokens
vocab = []

with open('./vocab/vocab.txt') as f:
    vocab = f.read().split() 
    
def clean_doc(doc):
    # split into tokens by white space
    tokens = word_tokenize(doc)
    
    # remove punctuation from each token
    tokens = [w.translate(remove_punctuation_table) for w in tokens]
    
    # remove stop words
    tokens = [w for w in tokens if w not in stop_words]
    
    # stemming
    porter = PorterStemmer()
    tokens = [porter.stem(w.lower()) for w in tokens]

    # filter out tokens not in vocab
    if len(vocab) > 0:
        tokens = [w for w in tokens if w in vocab]
    
    tokens = ' '.join(tokens)
    return tokens

### Multiprocessing

Pre-processing mentioned above requires heavy computation. 
To improve the speed, we parallelized the pre-processing using the *Pool module* in *multiprocessing package*.
Since we use a CPU having 8 cores, the size of Pool is set as 8.
By using this technique, **we could achieve 6~7 times speed up.** 
Using the single thread, it takes 10~12 minutes for cleaning up 12500 documents, whereas, using the multiple threads, it takes only 1 minute and 20~40 seconds.

In [5]:
# Serial version of clean_docs function
# def clean_docs(documents):
#     for doc in tqdm(documents):
#         clean_doc(doc)

# Parallel version of clean_docs function
def clean_docs(documents):
    # Since we use a CPU having 8 cores, the size of Pool is set as 8
    with Pool(8) as p:
        return list(tqdm(p.imap(clean_doc, documents), total=len(documents)))

print("Cleaning up for training-positive-docs")
cleaned_train_positive_docs = clean_docs(global_train_positive_docs)
print("Cleaning up for training-negative-docs")
cleaned_train_negative_docs = clean_docs(global_train_negative_docs)
print("Cleaning up for test-positive-docs")
cleaned_test_positive_docs = clean_docs(global_test_positive_docs)
print("Cleaning up for test-negative-docs")
cleaned_test_negative_docs = clean_docs(global_test_negative_docs) 

Cleaning up for training-positive-docs



Cleaning up for training-negative-docs



Cleaning up for test-positive-docs



Cleaning up for test-negative-docs


In [6]:
print(cleaned_test_negative_docs[0].split()[0:100])

['nt', 'realli', 'consid', 'conserv', 'nt', 'person', 'offend', 'film', 'pretti', 'clear', 'plot', 'character', 'film', 'secondari', 'messag', 'and', 'messag', 'conserv', 'either', 'evil', 'stupid', 'charact', 'either', 'good', 'american', 'brainless', 'greedi', 'evil', 'conserv', 'there', 'noth', 'clever', 'creativ', 'nt', 'realli', 'mind', 'polit', 'bia', 'nt', 'purpos', 'behind', 'movi', 'and', 'clearli', 'br', 'br', 'on', 'posit', 'side', 'cast', 'wonder', 'chri', 'cooper', 'impress', 'funni', 'first', 'two', 'three', 'time', 'old', 'joke', 'told', 'br', 'br', 'so', 'realli', 'hate', 'conserv', 'probabl', 'enjoy', 'film', 'look', 'someth', 'realist', 'charact', 'stori', 'less', 'better', 'watch', 'someth', 'els']
